# Lending Money with Spark

In [1]:
# Importing Libraries
import os
import sys
import math
import re
import urllib

import numpy as np
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession

from pyspark.sql.functions import regexp_replace, regexp_extract
from pyspark.sql.functions import col
from pyspark.sql.functions import count, mean, stddev_pop, min, max, avg
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import lit
from pyspark.sql.functions import coalesce
from pyspark.sql.functions import isnan, when, count, col, log
from pyspark.sql.types import FloatType

In [2]:
# Creating Spark Session
spark = SparkSession.builder.appName("LendingMoneySpark").getOrCreate()

In [3]:
# Importing Data

# File location and type
file_location = '../../../../Projects/lending_club/data/34_1110834_bundle_archive/loan.csv'
file_type = 'csv'

# CSV options
infer_schema = 'true'
first_row_is_header = 'true'
delimiter = ','

df = spark.read.format(file_type) \
    .option("multiline", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
	.option('inferSchema', infer_schema) \
	.option('header', first_row_is_header) \
	.option('sep', delimiter) \
	.load(file_location)

In [4]:
df.show(1) # Displaying first row

+----+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+--------+-----------+----------+----+----+------------------+------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+------------

In [5]:
df.printSchema() 

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: doubl

In [6]:
df.count()

2260668

In [7]:
# Dropping irrelevant columns
columns_to_drop = ['id', 'member_id', 'emp_title', 'url', 'desc', 'zip_code', 'title']
df = df.drop(*columns_to_drop)

In [8]:
df.printSchema()

root
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- mths_since_last_delinq: integer (nullable = true)
 |-- mths_since_last_record: integer (nullable = t

In [9]:
# Register the DataFrame as a SQL temporary view
temp_table = "loan"
df.createOrReplaceTempView(temp_table)

In [13]:
df.describe('loan_amnt').show()

+-------+------------------+
|summary|         loan_amnt|
+-------+------------------+
|  count|           2260668|
|   mean|15046.931227849467|
| stddev| 9190.245488233122|
|    min|               500|
|    max|             40000|
+-------+------------------+



In [14]:
quantileProbs = [0.25, 0.5, 0.75, 0.9]
relError = 0.00
df.stat.approxQuantile("loan_amnt", quantileProbs, relError)
# This indicates most of the loans were between 12900 and 20000

[8000.0, 12900.0, 20000.0, 30000.0]

In [15]:
quantileProbs = [0.25, 0.5, 0.75, 0.9]
relError = 0.00
df.stat.approxQuantile("funded_amnt", quantileProbs, relError)

[8000.0, 12875.0, 20000.0, 30000.0]

In [16]:
quantileProbs = [0.25, 0.5, 0.75, 0.9]
relError = 0.00
df.stat.approxQuantile("funded_amnt_inv", quantileProbs, relError)

[8000.0, 12800.0, 20000.0, 30000.0]

In [47]:
spark.sql("""
          SELECT YEAR(CAST(FROM_UNIXTIME(UNIX_TIMESTAMP(issue_d, 'MMM-yyyy')) AS date)) AS year, 
          COUNT(*) FROM loan GROUP BY year ORDER BY 2 DESC"""
         ).show()

# So, as you can see maximum loans were issued in 2018
# Also, loans were issued in incremental order
# I think we cannot claim this --> The loans applied by potential borrowers, the amount issued to the borrowers and the amount funded by investors are similarly distributed, meaning that it is most likely that qualified borrowers are going to get the loan they had applied for.

+----+--------+
|year|count(1)|
+----+--------+
|2018|  495242|
|2017|  443579|
|2016|  434407|
|2015|  421095|
|2014|  235629|
|2013|  134814|
|2012|   53367|
|2011|   21721|
|2010|   12537|
|2009|    5281|
|2008|    2393|
|2007|     603|
+----+--------+



In [48]:
#spark.sql("SELECT CAST(FROM_UNIXTIME(UNIX_TIMESTAMP(issue_d, 'MMM-yyyy')) AS date) AS issue_date FROM loan").show()

In [51]:
spark.sql("SELECT loan_status, COUNT(*) FROM loan GROUP BY loan_status ORDER BY 2 DESC").show(truncate=False)

+---------------------------------------------------+--------+
|loan_status                                        |count(1)|
+---------------------------------------------------+--------+
|Fully Paid                                         |1041952 |
|Current                                            |919695  |
|Charged Off                                        |261655  |
|Late (31-120 days)                                 |21897   |
|In Grace Period                                    |8952    |
|Late (16-30 days)                                  |3737    |
|Does not meet the credit policy. Status:Fully Paid |1988    |
|Does not meet the credit policy. Status:Charged Off|761     |
|Default                                            |31      |
+---------------------------------------------------+--------+



In [54]:
df=df.withColumn(
    "bad_loan", 
    when(df.loan_status.isin(["Charged Off", "Default", 
                                  "Does not meet the credit policy. Status:Charged Off", "In Grace Period", 
                                  "Late (16-30 days)", "Late (31-120 days)"
    ]),'Yes').otherwise('No')
)

In [55]:
temp_table = "loan"
df.createOrReplaceTempView(temp_table)

In [57]:
#df.printSchema()

In [61]:
spark.sql("SELECT bad_loan, COUNT(*)/(SELECT COUNT(*) FROM loan) AS per_bad FROM loan GROUP BY bad_loan").show()
# Total 13% bad loan and 87% good loan
# Remember we have considered current as good loan which can be removed for ML model

+--------+------------------+
|bad_loan|           per_bad|
+--------+------------------+
|      No|0.8686083051558212|
|     Yes|0.1313916948441788|
+--------+------------------+



In [94]:
spark.sql("SELECT bad_loan, COUNT(*) AS per_bad FROM loan GROUP BY bad_loan").show()

+--------+-------+
|bad_loan|per_bad|
+--------+-------+
|      No|1963635|
|     Yes| 297033|
+--------+-------+



In [62]:
west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']

In [64]:
df=df.withColumn(
    "region", 
    when(df.addr_state.isin(*west),'West')
    .when(df.addr_state.isin(*south_west),'SouthWest')
    .when(df.addr_state.isin(*south_east),'SouthEast')
    .when(df.addr_state.isin(*mid_west),'MidWest')
    .when(df.addr_state.isin(*north_east),'north_east')
)

In [66]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+-----------+---------------+----+--------+-----------+-----+---------+----------+--------------+----------+-------------------+-------+-----------+----------+-------+----------+----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------------+-----------+-

In [73]:
df=df.withColumn("term_cleaned",regexp_replace(col("term"), "months", "")) \
             .withColumn("emplen_cleaned",regexp_extract(col("emp_length"), "\\d+", 0))

In [74]:
df.printSchema()

root
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- mths_since_last_delinq: integer (nullable = true)
 |-- mths_since_last_record: integer (nullable = t

In [76]:
df = df.withColumn("term_cleaned",df["term_cleaned"].cast("int")) \
       .withColumn("emplen_cleaned",df["emplen_cleaned"].cast("int"))

In [77]:
temp_table = "loan"
df.createOrReplaceTempView(temp_table)

In [78]:
spark.sql("SELECT term, term_cleaned, emp_length, emplen_cleaned FROM loan").show()

+----------+------------+----------+--------------+
|      term|term_cleaned|emp_length|emplen_cleaned|
+----------+------------+----------+--------------+
| 36 months|          36| 10+ years|            10|
| 60 months|          60| 10+ years|            10|
| 36 months|          36|   6 years|             6|
| 36 months|          36| 10+ years|            10|
| 60 months|          60| 10+ years|            10|
| 36 months|          36| 10+ years|            10|
| 36 months|          36|   4 years|             4|
| 36 months|          36| 10+ years|            10|
| 36 months|          36| 10+ years|            10|
| 36 months|          36|  < 1 year|             1|
| 36 months|          36|  < 1 year|             1|
| 60 months|          60|   2 years|             2|
| 36 months|          36| 10+ years|            10|
| 36 months|          36|   4 years|             4|
| 60 months|          60| 10+ years|            10|
| 36 months|          36| 10+ years|            10|
| 36 months|

In [79]:
spark.sql("SELECT region, COUNT(*) FROM loan GROUP BY region ORDER BY 2 DESC").show()

+----------+--------+
|    region|count(1)|
+----------+--------+
| SouthEast|  550165|
|north_east|  519095|
|      West|  515469|
|   MidWest|  403150|
| SouthWest|  272789|
+----------+--------+



In [80]:
spark.sql("SELECT region, AVG(annual_inc) FROM loan GROUP BY region ORDER BY 2 DESC").show()

+----------+-----------------+
|    region|  avg(annual_inc)|
+----------+-----------------+
|north_east|82150.04021957604|
|      West|80199.99104900594|
| SouthWest|79772.06763062255|
| SouthEast|75036.60621253613|
|   MidWest|72646.06989445612|
+----------+-----------------+



In [81]:
spark.sql("SELECT region, AVG(int_rate) FROM loan GROUP BY region ORDER BY 2 DESC").show()

+----------+------------------+
|    region|     avg(int_rate)|
+----------+------------------+
| SouthEast|13.210704970371568|
|north_east|13.101527061562592|
|   MidWest| 13.05565399978571|
|      West|13.026665716117577|
| SouthWest| 13.01920301040458|
+----------+------------------+



In [82]:
spark.sql("SELECT region, AVG(emplen_cleaned) FROM loan GROUP BY region ORDER BY 2 DESC").show()

+----------+-------------------+
|    region|avg(emplen_cleaned)|
+----------+-------------------+
|   MidWest|  6.171929367545541|
|north_east|  6.098544670474444|
| SouthEast|  6.032337946389935|
|      West|  5.924145837636852|
| SouthWest|  5.813940146122143|
+----------+-------------------+



In [84]:
spark.sql("SELECT COUNT(*) from loan WHERE dti IS null").show()

+--------+
|count(1)|
+--------+
|    1711|
+--------+



In [85]:
spark.sql("SELECT application_type, dti, dti_joint FROM loan WHERE dti IS null").show()

+----------------+----+---------+
|application_type| dti|dti_joint|
+----------------+----+---------+
|       Joint App|null|    33.06|
|       Joint App|null|    17.67|
|       Joint App|null|     27.3|
|       Joint App|null|     8.74|
|       Joint App|null|    11.68|
|       Joint App|null|    28.01|
|       Joint App|null|    15.06|
|       Joint App|null|    12.79|
|       Joint App|null|     8.05|
|       Joint App|null|    24.22|
|       Joint App|null|    20.04|
|       Joint App|null|    19.01|
|       Joint App|null|    12.75|
|       Joint App|null|    22.61|
|       Joint App|null|    24.89|
|       Joint App|null|      0.6|
|       Joint App|null|    15.34|
|       Joint App|null|    23.69|
|       Joint App|null|    17.29|
|       Joint App|null|    29.25|
+----------------+----+---------+
only showing top 20 rows



In [86]:
df=df.withColumn("dti_cleaned",coalesce(col("dti"),col("dti_joint")))

In [87]:
temp_table = "loan"
df.createOrReplaceTempView(temp_table)

In [88]:
spark.sql("SELECT COUNT(*) from loan WHERE dti_cleaned IS null").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [90]:
df.stat.crosstab('loan_status','grade').show()

+--------------------+------+------+------+------+-----+-----+----+
|   loan_status_grade|     A|     B|     C|     D|    E|    F|   G|
+--------------------+------+------+------+------+-----+-----+----+
|     In Grace Period|   668|  2008|  3194|  1965|  794|  230|  93|
|          Fully Paid|212477|329075|286666|135839|56206|17220|4469|
|Does not meet the...|     8|    85|   148|   197|  158|   93|  72|
|             Default|     2|     6|     9|     8|    4|    2|   0|
|  Late (31-120 days)|  1314|  4758|  7640|  4985| 2226|  718| 256|
|   Late (16-30 days)|   267|   824|  1327|   829|  335|  114|  41|
|             Current|204435|275455|267326|120666|40174| 9006|2633|
|Does not meet the...|    90|   269|   481|   494|  378|  154| 122|
|         Charged Off| 13766| 51077| 83262| 59441|35364|14263|4482|
+--------------------+------+------+------+------+-----+-----+----+



In [91]:
df.stat.crosstab('region','loan_status').show()

+------------------+-----------+-------+-------+---------------------------------------------------+--------------------------------------------------+----------+---------------+-----------------+------------------+
|region_loan_status|Charged Off|Current|Default|Does not meet the credit policy. Status:Charged Off|Does not meet the credit policy. Status:Fully Paid|Fully Paid|In Grace Period|Late (16-30 days)|Late (31-120 days)|
+------------------+-----------+-------+-------+---------------------------------------------------+--------------------------------------------------+----------+---------------+-----------------+------------------+
|         SouthEast|      65460| 225937|      5|                                                184|                                               457|    249193|           2359|              974|              5596|
|         SouthWest|      31833| 110862|      4|                                                 79|                                    

In [92]:
df.stat.crosstab('bad_loan','grade').show()

+--------------+------+------+------+------+-----+-----+----+
|bad_loan_grade|     A|     B|     C|     D|    E|    F|   G|
+--------------+------+------+------+------+-----+-----+----+
|            No|417002|604799|554473|256999|96758|26380|7224|
|           Yes| 16025| 58758| 95580| 67425|38881|15420|4944|
+--------------+------+------+------+------+-----+-----+----+



In [93]:
df.stat.crosstab('region','bad_loan').show()

# May be dump this as another df and divide by total count to get the percentage

+---------------+------+-----+
|region_bad_loan|    No|  Yes|
+---------------+------+-----+
|      SouthEast|475587|74578|
|      SouthWest|236607|36182|
|        MidWest|352189|50961|
|     north_east|449402|69693|
|           West|449850|65619|
+---------------+------+-----+



In [95]:
spark.sql("SELECT AVG(int_rate) FROM loan").show()

+-----------------+
|    avg(int_rate)|
+-----------------+
|13.09291294419326|
+-----------------+



In [96]:
spark.sql("SELECT AVG(annual_inc) FROM loan").show()

+-----------------+
|  avg(annual_inc)|
+-----------------+
|77992.42868706738|
+-----------------+



In [99]:
spark.sql("SELECT addr_state, SUM(loan_amnt), AVG(int_rate), AVG(annual_inc) FROM loan GROUP BY addr_state ORDER BY 2 DESC LIMIT 5").show()

+----------+--------------+------------------+-----------------+
|addr_state|sum(loan_amnt)|     avg(int_rate)|  avg(annual_inc)|
+----------+--------------+------------------+-----------------+
|        CA|    4808480100|12.976628143969622|83847.98108853432|
|        TX|    2931133525|12.999443797465382|82728.81402050063|
|        NY|    2767160700| 13.26004973470032|81057.04801115961|
|        FL|    2333034500|13.164815329246071|73171.61167620425|
|        IL|    1410451950|12.960123611155202|79884.92210972556|
+----------+--------------+------------------+-----------------+



In [103]:
df=df.withColumn(
    "income_cat", 
    when(df["annual_inc"]<=100000,'Low')
    .when((df["annual_inc"] > 100000) & (df["annual_inc"] <= 200000) ,'Medium')
    .when(df["annual_inc"] > 200000,'High')
)

In [104]:
temp_table = "loan"
df.createOrReplaceTempView(temp_table)

In [105]:
df.printSchema()

root
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- mths_since_last_delinq: integer (nullable = true)
 |-- mths_since_last_record: integer (nullable = t

In [106]:
spark.sql("SELECT income_cat, AVG(loan_amnt) FROM loan GROUP BY income_cat ORDER BY 2 DESC ").show()

+----------+------------------+
|income_cat|    avg(loan_amnt)|
+----------+------------------+
|      High| 25830.78597917291|
|    Medium|21107.450458526873|
|       Low| 13413.56220061527|
|      null|            6300.0|
+----------+------------------+



In [107]:
# Remember to impute null values into loan_amnt column

In [112]:
df.stat.crosstab('income_cat','bad_loan').show()

+-------------------+-------+------+
|income_cat_bad_loan|     No|   Yes|
+-------------------+-------+------+
|             Medium| 353251| 40510|
|               High|  48782|  4610|
|                Low|1561598|251913|
|               null|      4|     0|
+-------------------+-------+------+



In [113]:
spark.sql("SELECT income_cat, AVG(int_rate) FROM loan GROUP BY income_cat ORDER BY 2 DESC").show()

+----------+------------------+
|income_cat|     avg(int_rate)|
+----------+------------------+
|       Low|13.307484459624483|
|    Medium| 12.29144595328553|
|      High| 11.71592204824534|
|      null| 7.827500000000001|
+----------+------------------+



In [114]:
spark.sql("SELECT income_cat, AVG(emplen_cleaned) FROM loan GROUP BY income_cat ORDER BY 2 DESC").show()

+----------+-------------------+
|income_cat|avg(emplen_cleaned)|
+----------+-------------------+
|    Medium|  6.449735805702042|
|      High|  6.339117613700806|
|       Low|  5.912311118569849|
|      null|                1.0|
+----------+-------------------+



In [116]:
spark.sql("SELECT addr_state, (SELECT SUM(loan_amnt) FROM loan WHERE bad_loan = 'Yes') FROM loan GROUP BY addr_state ORDER BY 2 DESC").show()

# Create same output as out 23 kaggle

+----------+----------------+
|addr_state|scalarsubquery()|
+----------+----------------+
|        DC|      4664104700|
|        MN|      4664104700|
|        SC|      4664104700|
|        AZ|      4664104700|
|        VA|      4664104700|
|        NJ|      4664104700|
|        OR|      4664104700|
|        KY|      4664104700|
|        WY|      4664104700|
|        LA|      4664104700|
|        NH|      4664104700|
|        CT|      4664104700|
|        NE|      4664104700|
|        CA|      4664104700|
|        WI|      4664104700|
|        VT|      4664104700|
|        RI|      4664104700|
|        MT|      4664104700|
|        MD|      4664104700|
|        ID|      4664104700|
+----------+----------------+
only showing top 20 rows



In [ ]:
# Generate output of 25, 26, 

In [117]:
# Maybe add interest_payment column?

In [118]:
spark.sql("SELECT purpose, COUNT(*) FROM loan GROUP BY purpose ORDER BY 2 DESC").show()

+------------------+--------+
|           purpose|count(1)|
+------------------+--------+
|debt_consolidation| 1277877|
|       credit_card|  516971|
|  home_improvement|  150457|
|             other|  139440|
|    major_purchase|   50445|
|           medical|   27488|
|    small_business|   24689|
|               car|   24013|
|          vacation|   15525|
|            moving|   15403|
|             house|   14136|
|           wedding|    2355|
|  renewable_energy|    1445|
|       educational|     424|
+------------------+--------+



In [119]:
# Stratified split in PySpark?

In [ ]:
####################################################################################################################3

In [7]:
# Register the DataFrame as a SQL temporary view
temp_table = "loan"
df.createOrReplaceTempView(temp_table)

In [8]:
spark.sql("SELECT * FROM loan")

DataFrame[id: string, member_id: string, loan_amnt: int, funded_amnt: int, funded_amnt_inv: double, term: string, int_rate: double, installment: double, grade: string, sub_grade: string, emp_title: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, issue_d: string, loan_status: string, pymnt_plan: string, url: string, desc: string, purpose: string, title: string, zip_code: string, addr_state: string, dti: double, delinq_2yrs: int, earliest_cr_line: string, inq_last_6mths: int, mths_since_last_delinq: int, mths_since_last_record: int, open_acc: int, pub_rec: int, revol_bal: int, revol_util: double, total_acc: int, initial_list_status: string, out_prncp: double, out_prncp_inv: double, total_pymnt: double, total_pymnt_inv: double, total_rec_prncp: double, total_rec_int: double, total_rec_late_fee: double, recoveries: double, collection_recovery_fee: double, last_pymnt_d: string, last_pymnt_amnt: double, next_pymnt_d: string, last_credit_pu

In [9]:
spark.sql("SELECT COUNT(*) FROM loan").show()

+--------+
|count(1)|
+--------+
| 2260668|
+--------+



In [10]:
df.describe().show()

KeyboardInterrupt: 

In [11]:
df_sel = df.select(
    "term", "home_ownership", "grade", "purpose", "int_rate", "installment",
    "addr_state","loan_status","application_type","loan_amnt","emp_length", 
    "annual_inc","dti","delinq_2yrs","revol_bal","revol_util","total_acc",
    "num_tl_90g_dpd_24m","dti_joint"
)

In [12]:
df_sel.describe().show()

+-------+----------+--------------+-------+-------+-----------------+-----------------+----------+------------------+----------------+------------------+----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|      term|home_ownership|  grade|purpose|         int_rate|      installment|addr_state|       loan_status|application_type|         loan_amnt|emp_length|        annual_inc|               dti|       delinq_2yrs|         revol_bal|        revol_util|         total_acc|num_tl_90g_dpd_24m|        dti_joint|
+-------+----------+--------------+-------+-------+-----------------+-----------------+----------+------------------+----------------+------------------+----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|   2260668|       2260668|2260668|2260668| 

In [12]:
df_sel.describe("term","loan_amnt","emp_length", "annual_inc","dti","delinq_2yrs","revol_util","total_acc").show()

+-------+----------+------------------+----------+------------------+------------------+-------------------+--------------------+--------------------+
|summary|      term|         loan_amnt|emp_length|        annual_inc|               dti|        delinq_2yrs|          revol_util|           total_acc|
+-------+----------+------------------+----------+------------------+------------------+-------------------+--------------------+--------------------+
|  count|   2260668|           2260668|   2260668|           2260664|           2258956|            2260638|             2258834|             2260618|
|   mean|      null|15046.931227849467|      null| 77992.44637764219|18.824907147958033|0.30721594910725347|    50.8582040171043|  24.455913212538377|
| stddev|      null| 9190.245488232787|      null|112696.22136091437|14.183773199305026| 0.8703772261110826|   185.3057416175736|  104.00261134157218|
|    min| 36 months|               500| reactors"|                 0|       Karen     "|      

In [13]:
df_sel.cache()

DataFrame[term: string, home_ownership: string, grade: string, purpose: string, int_rate: double, installment: double, addr_state: string, loan_status: string, application_type: string, loan_amnt: int, emp_length: string, annual_inc: double, dti: double, delinq_2yrs: int, revol_bal: int, revol_util: double, total_acc: int, num_tl_90g_dpd_24m: int, dti_joint: double]

In [14]:
df_sel.describe("loan_amnt","emp_length" ,"dti","delinq_2yrs","revol_util","total_acc").show()

+-------+------------------+----------+------------------+------------------+------------------+------------------+
|summary|         loan_amnt|emp_length|               dti|       delinq_2yrs|        revol_util|         total_acc|
+-------+------------------+----------+------------------+------------------+------------------+------------------+
|  count|           2260668|   2260668|           2258957|           2260639|           2258866|           2260639|
|   mean|15046.931227849467|      null| 18.82419643667267|0.3068791611575311|50.337696246702926|24.162552269513178|
| stddev| 9190.245488233122|      null|14.183328543078048|0.8672303328968344|24.713073320891283|11.987528317343706|
|    min|               500|    1 year|              -1.0|                 0|               0.0|                 1|
|    max|             40000|       n/a|             999.0|                58|             892.3|               176|
+-------+------------------+----------+------------------+--------------

In [15]:
spark.sql("SELECT DISTINCT(emp_length) FROM loan LIMIT 50").show()

+----------+
|emp_length|
+----------+
|   9 years|
|   5 years|
|    1 year|
|       n/a|
|   2 years|
|   7 years|
|   8 years|
|   4 years|
|   6 years|
|   3 years|
| 10+ years|
|  < 1 year|
+----------+



In [16]:
regex_string='years|year|\\+|\\<'
df_sel.select(regexp_replace(col("emp_length"), regex_string, "").alias("emplength_cleaned"),col("emp_length")).show(10)

+-----------------+----------+
|emplength_cleaned|emp_length|
+-----------------+----------+
|              10 | 10+ years|
|              10 | 10+ years|
|               6 |   6 years|
|              10 | 10+ years|
|              10 | 10+ years|
|              10 | 10+ years|
|               4 |   4 years|
|              10 | 10+ years|
|              10 | 10+ years|
|               1 |  < 1 year|
+-----------------+----------+
only showing top 10 rows



In [17]:
regex_string="\\d+"
df_sel.select(regexp_extract(col("emp_length"), regex_string, 0).alias("emplength_cleaned"),col("emp_length")).show(10)

+-----------------+----------+
|emplength_cleaned|emp_length|
+-----------------+----------+
|               10| 10+ years|
|               10| 10+ years|
|                6|   6 years|
|               10| 10+ years|
|               10| 10+ years|
|               10| 10+ years|
|                4|   4 years|
|               10| 10+ years|
|               10| 10+ years|
|                1|  < 1 year|
+-----------------+----------+
only showing top 10 rows



In [18]:
df_sel.show(2)

+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|  dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|
+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+
| 36 months|          RENT|    C|debt_consolidation|   13.56|      84.92|        NY|    Current|      Individual|     2500| 10+ years|   55000.0|18.24|          0|     4341|      10.3|       34|                 0|     null|
| 60 months|      MORTGAGE|    D|debt_consolidation|   18.94|     777.23|        LA|    Current|      In

In [19]:
df_sel=df_sel.withColumn("term_cleaned",regexp_replace(col("term"), "months", "")) \
             .withColumn("emplen_cleaned",regexp_extract(col("emp_length"), "\\d+", 0))

In [20]:
df_sel.select('term','term_cleaned','emp_length','emplen_cleaned').show(15)

+----------+------------+----------+--------------+
|      term|term_cleaned|emp_length|emplen_cleaned|
+----------+------------+----------+--------------+
| 36 months|         36 | 10+ years|            10|
| 60 months|         60 | 10+ years|            10|
| 36 months|         36 |   6 years|             6|
| 36 months|         36 | 10+ years|            10|
| 60 months|         60 | 10+ years|            10|
| 36 months|         36 | 10+ years|            10|
| 36 months|         36 |   4 years|             4|
| 36 months|         36 | 10+ years|            10|
| 36 months|         36 | 10+ years|            10|
| 36 months|         36 |  < 1 year|             1|
| 36 months|         36 |  < 1 year|             1|
| 60 months|         60 |   2 years|             2|
| 36 months|         36 | 10+ years|            10|
| 36 months|         36 |   4 years|             4|
| 60 months|         60 | 10+ years|            10|
+----------+------------+----------+--------------+
only showing

In [21]:
df_sel.printSchema()

root
 |-- term: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- num_tl_90g_dpd_24m: integer (nullable = true)
 |-- dti_joint: double (nullable = true)
 |-- term_cleaned: string (nullable = true)
 |-- emplen_cleaned: string (nullable = true)



In [22]:
table_name="loan_sel"
df_sel.createOrReplaceTempView(table_name)

In [23]:
spark.sql("SELECT * FROM loan_sel").show()

+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|  dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|
+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
| 36 months|          RENT|    C|debt_consolidation|   13.56|      84.92|        NY|    Current|      Individual|     2500| 10+ years|   55000.0|18.24|          0|     4341|      10.3|       34|                 0|     null|         36 |        

In [24]:
df_sel = df_sel.withColumn("annual_inc",df_sel["annual_inc"].cast("double")) # no need for this now

In [25]:
#df_sel.stat.cov('annual_inc', 'loan_amnt')

In [26]:
#result = df.describe()
#result.select(result['summary'],
#              format_number(result['Open'].cast('float'),2).alias('Open'),
#              format_number(result['High'].cast('float'),2).alias('High'),
#              format_number(result['Low'].cast('float'),2).alias('Low'),
#              format_number(result['Close'].cast('float'),2).alias('Close'),
#              result['Volume'].cast('int').alias('Volume')
#             ).show()

In [27]:
#df_sel.stat.corr('annual_inc', 'loan_amnt')

In [28]:
spark.sql("SELECT CORR(annual_inc, loan_amnt) AS corr FROM loan_sel").show()

+-------------------+
|               corr|
+-------------------+
|0.19724584986927352|
+-------------------+



In [29]:
df_sel.stat.crosstab('loan_status','grade').show()

+--------------------+------+------+------+------+-----+-----+----+
|   loan_status_grade|     A|     B|     C|     D|    E|    F|   G|
+--------------------+------+------+------+------+-----+-----+----+
|     In Grace Period|   668|  2008|  3194|  1965|  794|  230|  93|
|          Fully Paid|212477|329075|286666|135839|56206|17220|4469|
|Does not meet the...|     8|    85|   148|   197|  158|   93|  72|
|             Default|     2|     6|     9|     8|    4|    2|   0|
|  Late (31-120 days)|  1314|  4758|  7640|  4985| 2226|  718| 256|
|   Late (16-30 days)|   267|   824|  1327|   829|  335|  114|  41|
|             Current|204435|275455|267326|120666|40174| 9006|2633|
|Does not meet the...|    90|   269|   481|   494|  378|  154| 122|
|         Charged Off| 13766| 51077| 83262| 59441|35364|14263|4482|
+--------------------+------+------+------+------+-----+-----+----+



In [42]:
# Maybe add a plot here using matplotlib by dunping this df_sel into new df.

In [30]:
freq=df_sel.stat.freqItems(['purpose','grade'],0.3)

In [31]:
freq.collect()

[Row(purpose_freqItems=['debt_consolidation', 'credit_card'], grade_freqItems=['A', 'C', 'B'])]

In [32]:
spark.sql("SELECT DISTINCT(purpose) FROM loan_sel").show(50,truncate=False) # Some issue with DISTINCT Column, looks like csv isnot read properly 

+------------------+
|purpose           |
+------------------+
|wedding           |
|educational       |
|other             |
|small_business    |
|debt_consolidation|
|credit_card       |
|moving            |
|vacation          |
|renewable_energy  |
|house             |
|car               |
|major_purchase    |
|medical           |
|home_improvement  |
+------------------+



In [35]:
spark.sql("SELECT purpose, COUNT(*) AS count FROM loan_sel GROUP BY purpose").show()

+------------------+-------+
|           purpose|  count|
+------------------+-------+
|           wedding|   2355|
|       educational|    424|
|             other| 139440|
|    small_business|  24689|
|debt_consolidation|1277877|
|       credit_card| 516971|
|            moving|  15403|
|          vacation|  15525|
|  renewable_energy|   1445|
|             house|  14136|
|               car|  24013|
|    major_purchase|  50445|
|           medical|  27488|
|  home_improvement| 150457|
+------------------+-------+



In [36]:
spark.sql("SELECT purpose, COUNT(*) AS count FROM loan_sel GROUP BY purpose ORDER BY count DESC").show()

# INfer meaning our of this output

+------------------+-------+
|           purpose|  count|
+------------------+-------+
|debt_consolidation|1277877|
|       credit_card| 516971|
|  home_improvement| 150457|
|             other| 139440|
|    major_purchase|  50445|
|           medical|  27488|
|    small_business|  24689|
|               car|  24013|
|          vacation|  15525|
|            moving|  15403|
|             house|  14136|
|           wedding|   2355|
|  renewable_energy|   1445|
|       educational|    424|
+------------------+-------+



In [38]:
quantileProbs = [0.25, 0.5, 0.75, 0.9]
relError = 0.05
df_sel.stat.approxQuantile("loan_amnt", quantileProbs, relError)

[8000.0, 15000.0, 20000.0, 27000.0]

In [39]:
quantileProbs = [0.25, 0.5, 0.75, 0.9]
relError = 0.0
df_sel.stat.approxQuantile("loan_amnt", quantileProbs, relError)

[8000.0, 12900.0, 20000.0, 30000.0]

In [42]:
df_sel.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_sel.columns]).show()

+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
|term|home_ownership|grade|purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc| dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|
+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
|   0|             0|    0|      0|       0|          0|         0|          0|               0|        0|         0|         4|1711|         29|        0|      1802|       29|             70276|  2139962|           0|             0|
+----+--------------+-----+-------+--------+-----------+--------

In [45]:
spark.sql("SELECT loan_status, COUNT(*) FROM loan GROUP BY loan_status ORDER BY 2 DESC").show(truncate=False)

+---------------------------------------------------+--------+
|loan_status                                        |count(1)|
+---------------------------------------------------+--------+
|Fully Paid                                         |1041952 |
|Current                                            |919695  |
|Charged Off                                        |261655  |
|Late (31-120 days)                                 |21897   |
|In Grace Period                                    |8952    |
|Late (16-30 days)                                  |3737    |
|Does not meet the credit policy. Status:Fully Paid |1988    |
|Does not meet the credit policy. Status:Charged Off|761     |
|Default                                            |31      |
+---------------------------------------------------+--------+



In [46]:
df_sel=df_sel.na.drop("all", subset=["loan_status"])

In [47]:
df_sel.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_sel.columns]).show()

+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
|term|home_ownership|grade|purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc| dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|
+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
|   0|             0|    0|      0|       0|          0|         0|          0|               0|        0|         0|         4|1711|         29|        0|      1802|       29|             70276|  2139962|           0|             0|
+----+--------------+-----+-------+--------+-----------+--------

In [48]:
df_sel.describe("dti","revol_util").show()

+-------+------------------+------------------+
|summary|               dti|        revol_util|
+-------+------------------+------------------+
|  count|           2258957|           2258866|
|   mean| 18.82419643667267|50.337696246702926|
| stddev|14.183328543078048|24.713073320891283|
|    min|              -1.0|               0.0|
|    max|             999.0|             892.3|
+-------+------------------+------------------+



In [55]:
spark.sql("select ceil(REGEXP_REPLACE(revol_util,"\%","")), count(*) from loanstatus_sel group by ceil(REGEXP_REPLACE(revol_util,"\%",""))")

SyntaxError: unexpected character after line continuation character (<ipython-input-55-7d01aadf81ae>, line 1)

In [59]:
spark.sql("select * from loan_sel where revol_util is null").show()

+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|  dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|
+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
| 36 months|          RENT|    D|debt_consolidation|   17.97|      72.28|        IL|    Current|      Individual|     2000|   4 years|   51000.0|  2.4|          0|        0|      null|        9|                 0|     null|         36 |        

In [60]:
df_sel=df_sel.withColumn("revolutil_cleaned",regexp_extract(col("revol_util"), "\\d+", 0))

In [61]:
df_sel.describe('revol_util','revolutil_cleaned').show()

+-------+------------------+------------------+
|summary|        revol_util| revolutil_cleaned|
+-------+------------------+------------------+
|  count|           2258866|           2258866|
|   mean|50.337696246702926| 49.90740132438135|
| stddev|24.713073320891283|24.715936882582977|
|    min|               0.0|                 0|
|    max|             892.3|                99|
+-------+------------------+------------------+



In [62]:
def fill_avg(df, colname): 
    return df.select(colname).agg(avg(colname))

In [63]:
rev_avg=fill_avg(df_sel,'revolutil_cleaned')

In [65]:
rev_avg=fill_avg(df_sel,'revolutil_cleaned').first()[0]
df_sel=df_sel.withColumn('rev_avg',lit(rev_avg))

In [66]:
df_sel=df_sel.withColumn('revolutil_cleaned',coalesce(col('revolutil_cleaned'),col('rev_avg')))

In [67]:
df_sel.describe('revol_util','revolutil_cleaned').show()

+-------+------------------+------------------+
|summary|        revol_util| revolutil_cleaned|
+-------+------------------+------------------+
|  count|           2258866|           2260668|
|   mean|50.337696246702926| 49.90740132438078|
| stddev|24.713073320891283|24.706084260053252|
|    min|               0.0|                 0|
|    max|             892.3|                99|
+-------+------------------+------------------+



In [68]:
df_sel.count()

2260668

In [69]:
df_sel=df_sel.withColumn("revolutil_cleaned",df_sel["revolutil_cleaned"].cast("double"))

In [70]:
df_sel.describe('revol_util','revolutil_cleaned').show()

+-------+------------------+------------------+
|summary|        revol_util| revolutil_cleaned|
+-------+------------------+------------------+
|  count|           2258866|           2260668|
|   mean|50.337696246702926| 49.90740132438078|
| stddev|24.713073320891283|24.706084260053252|
|    min|               0.0|               0.0|
|    max|             892.3|             892.0|
+-------+------------------+------------------+



In [71]:
df_sel.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_sel.columns]).show()

+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-------+
|term|home_ownership|grade|purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc| dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|revolutil_cleaned|rev_avg|
+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-------+
|   0|             0|    0|      0|       0|          0|         0|          0|               0|        0|         0|         4|1711|         29|        0|      1802|       29|             70276|  2139962|           0|  

In [73]:
spark.sql("SELECT * from loan_sel WHERE dti IS null").show()

+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc| dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|
+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+
| 60 months|      MORTGAGE|    B|    major_purchase|   10.72|     280.84|        AZ|    Current|       Joint App|    13000|       n/a|       0.0|null|          0|    32720|      26.0|       47|                 0|    33.06|         60 |            

In [75]:
spark.sql("SELECT application_type, dti, dti_joint FROM loan WHERE dti IS null").show()

+----------------+----+---------+
|application_type| dti|dti_joint|
+----------------+----+---------+
|       Joint App|null|    33.06|
|       Joint App|null|    17.67|
|       Joint App|null|     27.3|
|       Joint App|null|     8.74|
|       Joint App|null|    11.68|
|       Joint App|null|    28.01|
|       Joint App|null|    15.06|
|       Joint App|null|    12.79|
|       Joint App|null|     8.05|
|       Joint App|null|    24.22|
|       Joint App|null|    20.04|
|       Joint App|null|    19.01|
|       Joint App|null|    12.75|
|       Joint App|null|    22.61|
|       Joint App|null|    24.89|
|       Joint App|null|      0.6|
|       Joint App|null|    15.34|
|       Joint App|null|    23.69|
|       Joint App|null|    17.29|
|       Joint App|null|    29.25|
+----------------+----+---------+
only showing top 20 rows



In [76]:
df_sel=df_sel.withColumn("dti_cleaned",coalesce(col("dti"),col("dti_joint")))

In [77]:
df_sel.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_sel.columns]).show()

+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-------+-----------+
|term|home_ownership|grade|purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc| dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|revolutil_cleaned|rev_avg|dti_cleaned|
+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-------+-----------+
|   0|             0|    0|      0|       0|          0|         0|          0|               0|        0|         0|         4|1711|         29|        0|      1802|       29|        

In [79]:
spark.sql("SELECT loan_status, COUNT(*) FROM loan_sel GROUP BY loan_status").show()

# Need to take care of default and two does not meet stuff. May be take a look at kaggle kernel to get idea what to do.
# Temporary fixing by dumping as good loans.

+--------------------+--------+
|         loan_status|count(1)|
+--------------------+--------+
|          Fully Paid| 1041952|
|             Default|      31|
|     In Grace Period|    8952|
|Does not meet the...|    1988|
|         Charged Off|  261655|
|  Late (31-120 days)|   21897|
|             Current|  919695|
|Does not meet the...|     761|
|   Late (16-30 days)|    3737|
+--------------------+--------+



In [80]:
df_sel.where(df_sel.loan_status.isin(["Late (31-120 days)", "Charged Off", "In Grace Period","Late (16-30 days)"])).show()

+----------+--------------+-----+------------------+--------+-----------+----------+------------------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|       loan_status|application_type|loan_amnt|emp_length|annual_inc|  dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|revolutil_cleaned|          rev_avg|dti_cleaned|
+----------+--------------+-----+------------------+--------+-----------+----------+------------------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+
| 36 months|      MORTGAGE|    A|debt_consolidation|    8.19|     659.13|      

In [82]:
df_sel=df_sel.withColumn(
    "bad_loan", 
    when(df_sel.loan_status.isin([
        "Late (31-120 days)", "Charged Off", "In Grace Period","Late (16-30 days)"
    ]),
         'Yes'
                    ).otherwise('No'))

In [83]:
df_sel.groupBy('bad_loan').count().show() # can use sql as well but need to dump into temp table first

+--------+-------+
|bad_loan|  count|
+--------+-------+
|      No|1964427|
|     Yes| 296241|
+--------+-------+



In [84]:
df_sel.filter(df_sel.bad_loan == 'Yes').show()

+----------+--------------+-----+------------------+--------+-----------+----------+------------------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|       loan_status|application_type|loan_amnt|emp_length|annual_inc|  dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|revolutil_cleaned|          rev_avg|dti_cleaned|bad_loan|
+----------+--------------+-----+------------------+--------+-----------+----------+------------------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
| 36 months|      MORTGAGE|    A|debt_consolidation|

In [85]:
df_sel.printSchema()

root
 |-- term: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- num_tl_90g_dpd_24m: integer (nullable = true)
 |-- dti_joint: double (nullable = true)
 |-- term_cleaned: string (nullable = true)
 |-- emplen_cleaned: string (nullable = true)
 |-- revolutil_cleaned: double (nullable = true)
 |-- rev_avg: double (nullable = false)
 |-- dti_cleaned: d

In [86]:
df_sel_final=df_sel.drop('revol_util','dti','dti_joint', 'rev_avg', 'term')

In [87]:
df_sel_final.printSchema()

root
 |-- home_ownership: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- num_tl_90g_dpd_24m: integer (nullable = true)
 |-- term_cleaned: string (nullable = true)
 |-- emplen_cleaned: string (nullable = true)
 |-- revolutil_cleaned: double (nullable = true)
 |-- dti_cleaned: double (nullable = true)
 |-- bad_loan: string (nullable = false)



In [88]:
df_sel_final.stat.crosstab('bad_loan', 'grade').show()

+--------------+------+------+------+------+-----+-----+----+
|bad_loan_grade|     A|     B|     C|     D|    E|    F|   G|
+--------------+------+------+------+------+-----+-----+----+
|            No|417012|604890|554630|257204|96920|26475|7296|
|           Yes| 16015| 58667| 95423| 67220|38719|15325|4872|
+--------------+------+------+------+------+-----+-----+----+



In [89]:
df_sel.describe('dti_cleaned').show()

+-------+------------------+
|summary|       dti_cleaned|
+-------+------------------+
|  count|           2260668|
|   mean|18.824178357899857|
| stddev|14.179782290150628|
|    min|              -1.0|
|    max|             999.0|
+-------+------------------+



In [90]:
df_sel.filter(df_sel.dti_cleaned > 100).show()

+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+------+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|   dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|revolutil_cleaned|          rev_avg|dti_cleaned|bad_loan|
+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+------+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
| 36 months|      MORTGAGE|    B|             other|   11.31|     328.

In [92]:
df_sel.filter(df_sel.dti_cleaned == -1).show()

# Maybe remove this two rows having dti = -1 which is not possible.

+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc| dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|revolutil_cleaned|          rev_avg|dti_cleaned|bad_loan|
+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
| 36 months|      MORTGAGE|    B|debt_consolidation|    9.75|     482.25|   

In [93]:
permanent_table_name = "lc_loan_data"

df_sel.write.format("parquet").saveAsTable(permanent_table_name) # dumping as real table

In [95]:
spark.sql("SELECT * FROM lc_loan_data").show()

+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|  dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|revolutil_cleaned|          rev_avg|dti_cleaned|bad_loan|
+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
| 36 months|          RENT|    C|debt_consolidation|   13.56|      84.92|

In [96]:
lc_df = spark.table('lc_loan_data') # Converting to spark dataframe 
lc_df.show()

+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
|      term|home_ownership|grade|           purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc|  dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|revolutil_cleaned|          rev_avg|dti_cleaned|bad_loan|
+----------+--------------+-----+------------------+--------+-----------+----------+-----------+----------------+---------+----------+----------+-----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-----------------+-----------+--------+
| 36 months|          RENT|    C|debt_consolidation|   13.56|      84.92|

In [97]:
lc_df.describe().show()

+-------+----------+--------------+-------+-------+-----------------+-----------------+----------+------------------+----------------+------------------+----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+--------+
|summary|      term|home_ownership|  grade|purpose|         int_rate|      installment|addr_state|       loan_status|application_type|         loan_amnt|emp_length|        annual_inc|               dti|       delinq_2yrs|         revol_bal|        revol_util|         total_acc|num_tl_90g_dpd_24m|        dti_joint|      term_cleaned|    emplen_cleaned| revolutil_cleaned|          rev_avg|       dti_cleaned|bad_loan|
+-------+----------+--------------+-------+-------+-----------------+-----------------+----------+------------------+----------------+------------------+---------

In [100]:
spark.sql("SELECT addr_state, SUM(loan_amnt) FROM lc_loan_data GROUP BY addr_state").show()

# Can dump this to pandas df and visualize it using matplotlib. 
# Or can we use it directly into matplotlib?

+----------+--------------+
|addr_state|sum(loan_amnt)|
+----------+--------------+
|        SC|     418481900|
|        AZ|     780966125|
|        LA|     382046275|
|        MN|     577447675|
|        NJ|    1316207975|
|        DC|      84707250|
|        OR|     379466875|
|        VA|    1013015875|
|        RI|     142960100|
|        KY|     313557625|
|        WY|      74153325|
|        NH|     166469500|
|        MI|     841646100|
|        NV|     470165300|
|        WI|     431766050|
|        ID|      62088425|
|        CA|    4808480100|
|        CT|     547556950|
|        NE|     110941450|
|        MT|      88170825|
+----------+--------------+
only showing top 20 rows



In [103]:
spark.sql("SELECT addr_state, COUNT(annual_inc) AS count FROM lc_loan_data GROUP BY addr_state").show()

+----------+------+
|addr_state| count|
+----------+------+
|        SC| 28003|
|        AZ| 53777|
|        LA| 25759|
|        MN| 39517|
|        NJ| 83132|
|        DC|  5356|
|        OR| 26789|
|        VA| 62954|
|        RI| 10005|
|        KY| 21887|
|        WY|  4748|
|        NH| 11142|
|        MI| 58770|
|        NV| 32657|
|        WI| 29877|
|        ID|  4308|
|        CA|314533|
|        CT| 35785|
|        NE|  7819|
|        MT|  6299|
+----------+------+
only showing top 20 rows



In [105]:
spark.sql("SELECT addr_state, COUNT(loan_amnt) FROM lc_loan_data WHERE bad_loan='Yes' GROUP BY addr_state").show()

+----------+----------------+
|addr_state|count(loan_amnt)|
+----------+----------------+
|        AZ|            7010|
|        SC|            2934|
|        LA|            3961|
|        MN|            5183|
|        NJ|           11290|
|        DC|             522|
|        OR|            2628|
|        VA|            8379|
|        RI|            1162|
|        KY|            2929|
|        WY|             544|
|        NH|            1064|
|        MI|            7838|
|        NV|            4780|
|        WI|            3639|
|        ID|             372|
|        CA|           42214|
|        CT|            3905|
|        NE|            1005|
|        MT|             699|
+----------+----------------+
only showing top 20 rows



In [107]:
spark.sql("SELECT grade, SUM(loan_amnt) AS tot_loan_amnt FROM lc_loan_data GROUP BY grade").show()

+-----+-------------+
|grade|tot_loan_amnt|
+-----+-------------+
|    F|    799410225|
|    E|   2367318100|
|    B|   9404817775|
|    D|   5097344375|
|    C|   9775551175|
|    A|   6323641900|
|    G|    248032375|
+-----+-------------+



In [109]:
spark.sql("SELECT grade, bad_loan, SUM(loan_amnt) AS tot_loan_amnt FROM lc_loan_data GROUP BY grade, bad_loan").show()

+-----+--------+-------------+
|grade|bad_loan|tot_loan_amnt|
+-----+--------+-------------+
|    G|     Yes|    100145100|
|    B|      No|   8593430200|
|    F|      No|    498421800|
|    E|     Yes|    704175725|
|    C|     Yes|   1430152275|
|    B|     Yes|    811387575|
|    A|     Yes|    224522800|
|    F|     Yes|    300988425|
|    C|      No|   8345398900|
|    A|      No|   6099119100|
|    E|      No|   1663142375|
|    D|     Yes|   1084992775|
|    G|      No|    147887275|
|    D|      No|   4012351600|
+-----+--------+-------------+



In [ ]:
#lc_df = lc_df.withColumn("exposure",when(lc_df.bad_loan=="No",col("revol_bal")).otherwise(-10*col("revol_bal")))
#display(lc_df)

In [ ]:
#display(lc_df.groupBy("bad_loan","grade").agg({"exposure": "sum"}))

In [110]:
lc_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in lc_df.columns]).show()

+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-------+-----------+--------+
|term|home_ownership|grade|purpose|int_rate|installment|addr_state|loan_status|application_type|loan_amnt|emp_length|annual_inc| dti|delinq_2yrs|revol_bal|revol_util|total_acc|num_tl_90g_dpd_24m|dti_joint|term_cleaned|emplen_cleaned|revolutil_cleaned|rev_avg|dti_cleaned|bad_loan|
+----+--------------+-----+-------+--------+-----------+----------+-----------+----------------+---------+----------+----------+----+-----------+---------+----------+---------+------------------+---------+------------+--------------+-----------------+-------+-----------+--------+
|   0|             0|    0|      0|       0|          0|         0|          0|               0|        0|         0|         4|1711|         29|        0|  

In [111]:
spark.sql("SELECT loan_amnt, bad_loan FROM lc_loan_data").show()

+---------+--------+
|loan_amnt|bad_loan|
+---------+--------+
|     2500|      No|
|    30000|      No|
|     5000|      No|
|     4000|      No|
|    30000|      No|
|     5550|      No|
|     2000|      No|
|     6000|      No|
|     5000|      No|
|     6000|      No|
|     5500|      No|
|    28000|      No|
|    11200|      No|
|     6500|      No|
|    22000|      No|
|     3500|      No|
|     7000|      No|
|    25000|      No|
|    16000|      No|
|    13000|      No|
+---------+--------+
only showing top 20 rows



In [113]:
spark.sql("SELECT loan_amnt FROM lc_loan_data WHERE bad_loan = 'Yes'").show()

+---------+
|loan_amnt|
+---------+
|    20975|
|    20000|
|     9100|
|     3525|
|    11000|
|     7000|
|    11500|
|     8000|
|    22000|
|    30000|
|     9500|
|    10000|
|    34000|
|    30000|
|    11200|
|     4850|
|    24000|
|    30000|
|    16000|
|     7400|
+---------+
only showing top 20 rows



In [114]:
def trim(string):
    return string.strip('%')

spark.udf.register("trimperct", trim) # Including as use defined function

<function __main__.trim(string)>

In [119]:
#spark.sql("SELECT int_rate, CAST(trimperct(int_rate) AS float) AS int_rate_float FROM lc_loan_data").show() # If it has % but I think we removed it. LEt's check in next line

In [118]:
spark.sql("SELECT int_rate FROM lc_loan_data").show()

+--------+
|int_rate|
+--------+
|   13.56|
|   18.94|
|   17.97|
|   18.94|
|   16.14|
|   15.02|
|   17.97|
|   13.56|
|   17.97|
|   14.47|
|   22.35|
|   11.31|
|    8.19|
|   17.97|
|   12.98|
|   16.14|
|   12.98|
|   16.91|
|   20.89|
|   14.47|
+--------+
only showing top 20 rows



In [121]:
spark.sql("SELECT bad_loan, int_rate FROM Lc_loan_data").show()

+--------+--------+
|bad_loan|int_rate|
+--------+--------+
|      No|   13.56|
|      No|   18.94|
|      No|   17.97|
|      No|   18.94|
|      No|   16.14|
|      No|   15.02|
|      No|   17.97|
|      No|   13.56|
|      No|   17.97|
|      No|   14.47|
|      No|   22.35|
|      No|   11.31|
|      No|    8.19|
|      No|   17.97|
|      No|   12.98|
|      No|   16.14|
|      No|   12.98|
|      No|   16.91|
|      No|   20.89|
|      No|   14.47|
+--------+--------+
only showing top 20 rows



In [123]:
spark.sql("SELECT bad_loan, installment FROM lc_loan_data").show()

+--------+-----------+
|bad_loan|installment|
+--------+-----------+
|      No|      84.92|
|      No|     777.23|
|      No|     180.69|
|      No|     146.51|
|      No|     731.78|
|      No|     192.45|
|      No|      72.28|
|      No|     203.79|
|      No|     180.69|
|      No|     206.44|
|      No|     211.05|
|      No|     613.13|
|      No|     351.95|
|      No|      234.9|
|      No|     500.35|
|      No|      123.3|
|      No|      235.8|
|      No|     620.11|
|      No|     431.87|
|      No|     305.67|
+--------+-----------+
only showing top 20 rows



In [125]:
spark.sql("SELECT home_ownership, bad_loan, loan_amnt FROM lc_loan_data").show()

+--------------+--------+---------+
|home_ownership|bad_loan|loan_amnt|
+--------------+--------+---------+
|          RENT|      No|     2500|
|      MORTGAGE|      No|    30000|
|      MORTGAGE|      No|     5000|
|      MORTGAGE|      No|     4000|
|      MORTGAGE|      No|    30000|
|      MORTGAGE|      No|     5550|
|          RENT|      No|     2000|
|          RENT|      No|     6000|
|      MORTGAGE|      No|     5000|
|           OWN|      No|     6000|
|      MORTGAGE|      No|     5500|
|      MORTGAGE|      No|    28000|
|      MORTGAGE|      No|    11200|
|      MORTGAGE|      No|     6500|
|      MORTGAGE|      No|    22000|
|      MORTGAGE|      No|     3500|
|      MORTGAGE|      No|     7000|
|      MORTGAGE|      No|    25000|
|      MORTGAGE|      No|    16000|
|      MORTGAGE|      No|    13000|
+--------------+--------+---------+
only showing top 20 rows



In [127]:
spark.sql("SELECT grade, purpose, COUNT(*) AS count FROM lc_loan_data GROUP BY grade, purpose").show()

+-----+----------------+------+
|grade|         purpose| count|
+-----+----------------+------+
|    B|          moving|  3402|
|    G|         wedding|    26|
|    D|     credit_card| 48764|
|    F|home_improvement|  2514|
|    F|          moving|   530|
|    D|renewable_energy|   298|
|    D|         wedding|   506|
|    B|         wedding|   554|
|    C|renewable_energy|   419|
|    A|     credit_card|140508|
|    G|         medical|   132|
|    E|renewable_energy|   151|
|    F|           house|   597|
|    A|         medical|  4459|
|    A|         wedding|   449|
|    C|          moving|  4785|
|    E|         wedding|   221|
|    B|     educational|   112|
|    E|         medical|  1919|
|    D|  major_purchase|  6875|
+-----+----------------+------+
only showing top 20 rows



In [128]:
spark.sql("SELECT grade, loan_status, COUNT(loan_status) FROM lc_loan_data GROUP BY grade, loan_status").show()

+-----+--------------------+------------------+
|grade|         loan_status|count(loan_status)|
+-----+--------------------+------------------+
|    E|          Fully Paid|             56206|
|    A|Does not meet the...|                90|
|    E|             Default|                 4|
|    B|   Late (16-30 days)|               824|
|    A|     In Grace Period|               668|
|    C|  Late (31-120 days)|              7640|
|    C|             Default|                 9|
|    F|  Late (31-120 days)|               718|
|    G|Does not meet the...|                72|
|    D|             Default|                 8|
|    C|          Fully Paid|            286666|
|    B|             Default|                 6|
|    E|   Late (16-30 days)|               335|
|    F|          Fully Paid|             17220|
|    E|     In Grace Period|               794|
|    F|   Late (16-30 days)|               114|
|    A|             Current|            204435|
|    G|     In Grace Period|            

In [130]:
spark.sql("SELECT bad_loan, count(*) FROM lc_loan_data GROUP BY bad_loan").show()

+--------+--------+
|bad_loan|count(1)|
+--------+--------+
|      No| 1964427|
|     Yes|  296241|
+--------+--------+



In [132]:
spark.sql("SELECT installment, loan_amnt, bad_loan FROM lc_loan_data").show()

+-----------+---------+--------+
|installment|loan_amnt|bad_loan|
+-----------+---------+--------+
|      84.92|     2500|      No|
|     777.23|    30000|      No|
|     180.69|     5000|      No|
|     146.51|     4000|      No|
|     731.78|    30000|      No|
|     192.45|     5550|      No|
|      72.28|     2000|      No|
|     203.79|     6000|      No|
|     180.69|     5000|      No|
|     206.44|     6000|      No|
|     211.05|     5500|      No|
|     613.13|    28000|      No|
|     351.95|    11200|      No|
|      234.9|     6500|      No|
|     500.35|    22000|      No|
|      123.3|     3500|      No|
|      235.8|     7000|      No|
|     620.11|    25000|      No|
|     431.87|    16000|      No|
|     305.67|    13000|      No|
+-----------+---------+--------+
only showing top 20 rows



In [133]:
lc_df.stat.corr('installment', 'loan_amnt')

0.9456267528524241

In [137]:
pd_df=lc_df.toPandas()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35561)
Traceback (most recent call last):
  File "/home/venom/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/venom/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:35561)

In [138]:
# Fix above error

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.clf()
sns.distplot(pd_df.loc[pd_df['dti'].notnull() & (pd_df['dti']<50), 'dti'])
plt.xlabel('dti')
plt.ylabel('count')
display()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.clf()
sns.countplot(pd_df.loc[pd_df['total_acc']<120,'total_acc'],order=sorted(pd_df['total_acc'].unique()), saturation=1)
_, _ = plt.xticks(np.arange(0, 120, 10), np.arange(0, 120, 10))
plt.xlabel('total_acc')
plt.ylabel('count')
display()

In [ ]:
##################### Kaggle Kernel